In [1]:
import cira

cira.auth.KEY_FILE = "../../alpc_key.json"
assert cira.auth.check_keys(), "the set keys dose not work"

In [2]:
portfolio = cira.Portfolio()

In [3]:
import pandas as pd 
import numpy as np
from typing import List, Dict

In [4]:
from datetime import datetime

assets_symbols = ["MSFT", "TSLA", "AMZN"]
stk_hist_data = {}
IS_CACHED = True

for SYMBOL in assets_symbols:
    SYM_HIST_FILE = f"./{SYMBOL}.csv"

    stk = cira.Stock(SYMBOL)

    if not IS_CACHED:
        start = datetime(2015, 7, 1)
        end = datetime(2023, 7, 1)
        stk.save_historical_data(SYM_HIST_FILE, start, end)
    
    data = stk.load_historical_data(SYM_HIST_FILE)
    stk_hist_data[SYMBOL] = data



In [5]:

df = pd.DataFrame()
X = pd.DataFrame()

for sym, hist in stk_hist_data.items():
    df[sym] = hist["close"] - hist["open"]
    df[sym] = [1 if df[sym].iloc[i] > 0 else 0 for i in range(len(df))]
    hist.drop("close", axis=1, inplace=True)
    hist.drop("symbol", axis=1, inplace=True)

    hist_keys = {k:f"{sym}_{k}" for k in hist.keys()}
    hist.rename(columns=hist_keys, inplace=True)
    X = pd.concat([X, hist], axis=1)

df["CASH"] = 0

X.head()

,MSFT_open,MSFT_high,MSFT_low,MSFT_volume,MSFT_trade_count,MSFT_vwap,TSLA_open,TSLA_high,TSLA_low,TSLA_volume,TSLA_trade_count,TSLA_vwap,AMZN_open,AMZN_high,AMZN_low,AMZN_volume,AMZN_trade_count,AMZN_vwap
timestamp,,,,,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,48.18,48.61,47.36,59379610.0,272783.0,48.05,15.38,15.43,14.60,104264850.0,69017.0,14.91,32.81,32.89,31.38,192327900.0,137283.0,31.97
2016-01-05 05:00:00+00:00,48.72,49.13,48.38,36457804.0,180639.0,48.83,15.09,15.13,14.67,49003380.0,31302.0,14.88,32.34,32.35,31.38,119738940.0,72594.0,31.75
2016-01-06 05:00:00+00:00,48.18,48.25,47.58,41899187.0,225858.0,47.99,14.67,14.67,14.40,58041030.0,33013.0,14.52,31.10,31.99,31.02,109533180.0,67908.0,31.58
2016-01-07 05:00:00+00:00,46.75,47.44,46.19,61897908.0,303273.0,46.64,14.28,14.56,14.24,54639360.0,33419.0,14.40,31.09,31.50,30.26,145573100.0,92501.0,30.81
2016-01-08 05:00:00+00:00,46.45,47.26,46.26,52260304.0,261924.0,46.74,14.52,14.70,14.05,56049465.0,32657.0,14.31,30.98,31.21,30.30,114584160.0,71586.0,30.74


In [6]:
df

,MSFT,TSLA,AMZN,CASH
timestamp,,,,
2016-01-04 05:00:00+00:00,1,0,0,0
2016-01-05 05:00:00+00:00,1,0,0,0
2016-01-06 05:00:00+00:00,0,0,1,0
2016-01-07 05:00:00+00:00,0,1,0,0
2016-01-08 05:00:00+00:00,0,0,0,0
...,...,...,...,...
2023-06-26 04:00:00+00:00,0,0,0,0
2023-06-27 04:00:00+00:00,1,1,1,0
2023-06-28 04:00:00+00:00,1,1,1,0


In [7]:
split_prc = 0.8
X_train = X.iloc[:int(len(X)*split_prc)]
X_test =  X.iloc[int(len(X)*split_prc):]
print(len(X), len(X_train), len(X_test))
assert len(X) == len(X_train) + len(X_test)

y_train = df.iloc[:int(len(df)*split_prc)]
y_test =  df.iloc[int(len(df)*split_prc):]
print(len(df), len(y_train), len(y_test))
assert len(df) == len(y_train) + len(y_test)
assert len(X_train) == len(y_train)

1886 1508 378
1886 1508 378


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


model = SVC(kernel='rbf', gamma=0.12)

model.fit(X_train.values, y_train.values)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

ValueError: y should be a 1d array, got an array of shape (1508, 4) instead.